In [1]:
import sqlite3
import pandas as pd

import sys
sys.path.append("../")
from controller import *

In [2]:
decretos_sen = load_senado_decrees('../Senado API files/', '../radar_db')
classes_sen=get_classes("../supervised NLP/Classification/classificacao senado")
classes_sen_names = [c[0] for c in classes_sen]
decretos_sen = multilabel_classify_decrees(decretos_sen, classes_sen)
decretos_sen = get_df_senado(decretos_sen)
decretos_sen = decretos_sen.set_index("numero ato")

decretos_minist = load_ministerio_decrees('../radar_db')
classes_minist=get_classes("../supervised NLP/Classification/classificacao referenda")
classes_minist_names = [c[0] for c in classes_minist]
decretos_minist = multilabel_classify_decrees(decretos_minist, classes_minist)
decretos_minist = get_df_ministerio(decretos_minist)
decretos_minist['cod_ident_ato'] = [int(d.replace(" ", "/").split("/")[1].replace('.', '')) for d in decretos_minist['cod_ident_ato']]
decretos_minist = decretos_minist.set_index("cod_ident_ato")

10135 is not classified by Senado.
10125 is not classified by Senado.
10021 is not classified by Senado.
9205 is not classified by Senado.
8420 is not classified by Senado.


In [3]:
decretos_sen

,full_text,classificação senado
numero ato,,
10095,Dispõe sobre o Comitê Consultivo de Nanotecnol...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]"
10093,"Altera o Decreto nº 8.642 , de 19 de janeiro ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
10091,Promulga o Acordo entre o Governo da República...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10090,Promulga o Acordo entre o Governo da República...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10094,Dispõe sobre o Comitê Interministerial de Tecn...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]"
...,...,...
3332,"PROMULGA O PROTOCOLO AO TRATADO DE AMIZADE, CO...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3337,PRORROGA A VIGÊNCIA DOS RESTOS A PAGAR QUE ESP...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]"
3328,"ALTERA O DECRETO N. 2.889, DE 21/12/1998, QUE ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"


In [14]:
nodes = pd.read_csv('graph csv/nodes_gephi_v3.csv')
nodes['Label'] = [l.replace(" ", "/").split("/")[1].replace('.', '') for l in nodes['Label']]
# sorting test
nodes.sort_values(["degree", "outdegree"], ascending=[False, False])

,Id,Label,timeset,indegree,outdegree,degree,modularity_class,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality
1214,DEC 3.500/2000,3500,NaN,16,0,16,356,0,0.000000,0.000000,0.000000
2209,DEC 8.325/2014,8325,NaN,0,15,15,222,3,0.651163,0.755952,0.000000
1155,DEC 9.000/2017,9000,NaN,1,14,15,108,13,0.199708,0.310471,0.000010
1694,DEC 8.578/2015,8578,NaN,3,12,15,356,16,0.166667,0.237017,0.000048
215,DEC 9.579/2018,9579,NaN,5,10,15,72,3,0.558824,0.710526,0.000004
1162,DEC 9.038/2017,9038,NaN,6,9,15,9,10,0.240929,0.322050,0.000109


In [16]:
def count_nodes(decretos, nodes, metric, classification, limit, classes_names):
    central_nodes = nodes[nodes[metric] != 0.0].sort_values([metric, "outdegree"], ascending=[False, False])[:limit]
    counters = np.zeros(len(classes_names)+1)
    for d in central_nodes[["Label", metric]].values:
        counters = np.add(counters, decretos.loc[int(d[0])][classification])
        
    for c in range(len(counters)):
        try:
            print(classes_names[c], counters[c]/limit*100)
        except:
            print("Unknown: ", counters[c]/limit*100)
            print("\n")

In [17]:
print("BETWEENESS:")
print("Senado")
count_nodes(decretos_sen, nodes, "betweenesscentrality", "classificação senado", 100, classes_sen_names)
print("Referenda")
count_nodes(decretos_minist, nodes, "betweenesscentrality", "referenda", 100, classes_minist_names)

print("CLOSENESS:")
print("Senado")
count_nodes(decretos_sen, nodes, "closnesscentrality", "classificação senado", 100, classes_sen_names)
print("Referenda")
count_nodes(decretos_minist, nodes, "closnesscentrality", "referenda", 100, classes_minist_names)

print("DEGREE:")
print("Senado")
count_nodes(decretos_sen, nodes, "degree", "classificação senado", 100, classes_sen_names)
print("Referenda")
count_nodes(decretos_minist, nodes, "degree", "referenda", 100, classes_minist_names)

BETWEENESS:
Senado
Saúde 0.0
Relações Exteriores 1.0
Meio ambiente 0.0
Educação, cultura e esporte 0.0
Segurança Pública e Defesa 0.0
Trabalho e Previdência 4.0
Agricultura, pecuária e pesca 0.0
Ciência, tecnologia e comunicações 1.0
Social 0.0
Indústria, comércio, turismo, transporte/transporte de mercadorias 0.0
Economia, planejamento e sistema financeiro 1.0
Assuntos internos,  cargos e comissões, Estado 88.0
Tributos 6.0
Minas e Energia 0.0
Justiça e direitos 2.0
Unknown:  2.0


Referenda
Saúde 0.0
Relações Exteriores 2.0
Meio ambiente 1.0
Educação, cultura e esporte 5.0
Justiça e Segurança 16.0
Trabalho e Previdência 6.0
Transporte/transporte de mercadorias 0.0
Agricultura, pecuária e pesca 3.0
Ciência e tecnologia 5.0
Social 4.0
Presidência 18.0
Economia e planejamento 96.0
Indústria, comércio, obras públicas, turismo 4.0
Unknown:  0.0


CLOSENESS:
Senado
Saúde 3.0
Relações Exteriores 8.0
Meio ambiente 1.0
Educação, cultura e esporte 5.0
Segurança Pública e Defesa 7.0000000000000